In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Veri setini yükle
train_data = pd.read_csv('/data/mnist_train.csv')
test_data = pd.read_csv('/data/mnist_test.csv')

# Veriyi küçült
train_data = train_data.sample(frac=0.2, random_state=1)
test_data = test_data.sample(frac=0.2, random_state=1)

# Bağımsız değişkenleri ve hedef değişkeni ayır
X_train = train_data.drop('label', axis=1).values
y_train = to_categorical(train_data['label'].values)

X_test = test_data.drop('label', axis=1).values
y_test = to_categorical(test_data['label'].values)

# Veriyi normalize et
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# Giriş boyutlarını yeniden boyutlandır ve genişlet
X_train_reshaped = X_train.reshape(-1, 28, 28, 1)
X_test_reshaped = X_test.reshape(-1, 28, 28, 1)
X_train_resized = tf.image.resize(X_train_reshaped, (224, 224))
X_test_resized = tf.image.resize(X_test_reshaped, (224, 224))
X_train_resized_rgb = np.stack((X_train_resized[:, :, :, 0],) * 3, axis=-1)
X_test_resized_rgb = np.stack((X_test_resized[:, :, :, 0],) * 3, axis=-1)

# VGG modelini oluştur
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_vgg = models.Sequential()
model_vgg.add(base_model)
model_vgg.add(layers.Flatten())
model_vgg.add(layers.Dense(256, activation='relu'))
model_vgg.add(layers.Dropout(0.5))
model_vgg.add(layers.Dense(10, activation='softmax'))
model_vgg.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli eğit
model_vgg.fit(X_train_resized_rgb, y_train, epochs=5, batch_size=64, validation_split=0.2)

# Modeli değerlendir
y_pred_vgg = model_vgg.predict(X_test_resized_rgb)
y_pred_vgg_classes = np.argmax(y_pred_vgg, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

accuracy_vgg = accuracy_score(y_test_classes, y_pred_vgg_classes)
precision_vgg = precision_score(y_test_classes, y_pred_vgg_classes, average='weighted')
recall_vgg = recall_score(y_test_classes, y_pred_vgg_classes, average='weighted')
f1_vgg = f1_score(y_test_classes, y_pred_vgg_classes, average='weighted')

print(f"Accuracy: {accuracy_vgg}")
print(f"Precision: {precision_vgg}")
print(f"Recall: {recall_vgg}")
print(f"F1 Score: {f1_vgg}")


58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/5
41/41 [==============================] - 84s 1s/step - loss: 2.4667 - accuracy: 0.1114 - val_loss: 2.3025 - val_accuracy: 0.0872
Epoch 2/5
41/41 [==============================] - 32s 781ms/step - loss: 2.3038 - accuracy: 0.0970 - val_loss: 2.3035 - val_accuracy: 0.0872
Epoch 3/5
41/41 [==============================] - 32s 774ms/step - loss: 2.3010 - accuracy: 0.1075 - val_loss: 2.3034 - val_accuracy: 0.0872
Epoch 4/5
41/41 [==============================] - 32s 769ms/step - loss: 2.2999 - accuracy: 0.1145 - val_loss: 2.3038 - val_accuracy: 0.1184
Epoch 5/5
63/63 [==============================] - 17s 176ms/step
Accuracy: 0.113
Precision: 0.012769
Recall: 0.113
F1 Score: 0.02294519317160827


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
